# Promise

## The `Promise` object is used for asynchronous computations

## A `Promise` represents a value which may be available now, or in the future, or never

<img src="./assets/promises----promise_states_simple----revised.jpg" />

# `new Promise()`

## The constructor `Promise()` must be used with `new`, and must be provided a function callback

## This function is passed two function callbacks that act as resolution capabilities for the _Promise_, commonly denoted as `resolve()` and `reject()`

In [16]:
{

const p = new Promise((resolve, reject) => {
    console.log('Promise pending.');
});

}

Promise pending.


## `Promise.prototype.then()` takes one or two parameters: the fulfillment callback and the rejection callback

In [2]:
{

const p = new Promise((resolve, reject) => {
    console.log('Promise pending.');
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise pending.


## `reject()` simply rejects the _Promise_

In [3]:
{

const p = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise rejected.


## `resolve()` can either fulfill or reject the _Promise_

<img src="./assets/promises----promise_states_all----revised.jpg" />

## If `resolve()` is passed a non-_Promise_ value, then the instantiating _Promise_ is fulfilled with that value

In [6]:
{

const p = new Promise(resolve => {
    resolve(42);
})
.then(
    value => {
        console.log(value);
    }
);

}

42


## If `resolve()` is passed a _Promise_, then the resolution of the passed _Promise_ will be adopted

In [7]:
{

const fulfilledPromise = new Promise(resolve => {
    resolve();
});
    
const rejectedPromise = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
});

const p1 = new Promise(resolve => {
    resolve(fulfilledPromise);
})
.then(
    () => {
        console.log('First  Promise fulfilled.')
    },
    () => {
        console.log('First  Promise rejected.')
    }
);

const p2 = new Promise(resolve => {
    resolve(rejectedPromise);
})
.then(
    () => {
        console.log('Second Promise fulfilled.')
    },
    () => {
        console.log('Second Promise rejected.')
    }
);

}

First  Promise fulfilled.
Second Promise rejected.


# `Promise.prototype.then()` & `Promise.prototype.catch()`

## If `resolve()` or `reject()` is called multiple times, or both are called, the _Promise_ will accept only the first resolution, and silently ignore any subsequent attempts

In [8]:
{

const p = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
    setTimeout(reject, 100);
    setTimeout(resolve, 200);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise rejected.


## If an exception is thrown at any point in the creation of a _Promise_, that exception will be caught and it will force the _Promise_ to reject

In [3]:
/* RUN IN BABEL KERNEL */
{

const p = new Promise(resolve => {
    let str = '';
    str.push(0);
    resolve(str);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    (e) => {
        console.log(`Promise rejected. ${e}.`);
    }
);

}

'use strict'

Promise rejected. TypeError: str.push is not a function.


## If you call `resolve()` or `reject()` with multiple parameters, all subsequent parameters beyond the first will be silently ignored

In [1]:
{

const p = new Promise(resolve => {
    resolve('a', 'b');
})
.then(
    (arg0, arg1) => {
        console.log(`arg0: '${arg0}'`);
        console.log(`arg1:  ${arg1}`);
    }
);

}

arg0: 'a'
arg1:  undefined


## `Promise.prototype.catch()` takes only the rejection callback as a parameter, and automatically substitutes the default fulfillment callback

In [4]:
{
    
const p1 = new Promise(resolve => {
    resolve();
})
.catch(
    (e) => {
        console.log(`First  Promise rejected. ${e}.`);
    }
);

const p2 = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
})
.catch(
    () => {
        console.log('Second Promise rejected.');
    }
);

}

Second Promise rejected.


## `Promise.prototype.then()` and `Promise.prototype.catch()` return a new _Promise_

In [1]:
{

const p = new Promise(resolve => {
    resolve();
});

const thenPromise = p.then(() => {});

const catchPromise = p.catch(() => {});

console.log(`then():  ${thenPromise}`);
console.log(`catch(): ${catchPromise}`);

}

then():  [object Promise]
catch(): [object Promise]


# `Promise.resolve()`

## Returns a _Promise_ that's fulfilled with that value

In [6]:
{

const p1 = Promise.resolve(42);

const p2 = p1.then(
    value => {
        console.log(value);
    }
);

}

42


## If you pass a _Promise_ to `Promise.resolve()`, you get the same _Promise_ back

In [7]:
{

const p1 = new Promise(resolve => {
    resolve(42);
});

const p2 = Promise.resolve(p1);
    
const p3 = p2.then(
    value => {
        console.log(value);
        console.log(`p1 and p2 are the same? ${p1 === p2}`);
    }
);

}

42
p1 and p2 are the same? true


## If the resolution handler returns a non-_Promise_ value, the new returned _Promise_ is fulfilled

In [8]:
{

const p = new Promise(resolve => {
    resolve();
})
.then(
    () => {
        return 42; // Equivalent to: return Promise.resolve(42);
    }
)
.then(
    value => {
        console.log(value)
    }
);

}

42


## If the resolution handler returns a _Promise_, its resolution will be adopted

In [11]:
{

const p = new Promise(resolve => {
    resolve();
})
.then(
    () => {
        return new Promise((resolve, reject) => {
            setTimeout(reject, 0);
        });
    }
)
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise rejected.


## The _Promise_ returned from the resolution handler does _not_ inherit the resolution state of its chained _Promise_

In [19]:
{

const p = new Promise((resolve, reject) => {
    setTimeout(reject, 0);
})
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
)
.then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.')
    }
);

}

Promise rejected.
Promise fulfilled.


## If an exception is thrown in the resolution handler, the new returned _Promise_ is rejected

In [21]:
{

const p = new Promise(resolve => {
    resolve();
})
.then(() => {
    var str = '';
    str.push(0);
    return str;
})
.then(
    () => {
        console.log('Promise fulfilled.')
    },
    (e) => {
        console.log(`Promise rejected. ${e}.`)
    }
);

}

Promise rejected. TypeError: str.push is not a function.


# `Promise.reject()`

## Returns a rejected _Promise_

In [2]:
/* RUN IN BABEL KERNEL */
{

const p = Promise.reject();

p.then(
    () => {
        console.log('Promise fulfilled.')
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

Promise { <pending> }

Promise rejected.


# `Promise.all()`

## `Promise.all()` expects a single argument: an array of _Promise_ instances

## The _Promise_ returned from `Promise.all()` will receive a fulfillment argument that is an array of all the fulfillment values from the passed in _Promises_, in the same order as specified

In [3]:
{

const p1 = new Promise(resolve => {
    resolve(1);
});

const p2 = new Promise(resolve => {
    resolve(2);
});

const p3 = Promise.all([p1, p2]).then(
    values => {
        console.log(values)
    }
);

}

[Array] [1,2]


## If any one of those _Promise_ instances rejects, the main `Promise.all()` _Promise_ is immediately rejected, discarding all results from any other _Promise_ instances

In [4]:
{

const p1 = new Promise(resolve => {
    resolve();
});

const p2 = new Promise((resolve, reject) => {
    setTimeout(() => reject('Error 42'), 0);
});

const p3 = Promise.all([p1, p2]).then(
    () => {
        console.log('Promise fulfilled.');
    },
    (e) => {
        console.log(`Promise rejected. ${e}.`);
    }
);

}

Promise rejected: Error 42.


## If `Promise.all()` is passed an empty array, it will fulfill immediately

In [6]:
{

const p = Promise.all([]).then(
    () => {
        console.log('Promise fulfilled.');
    },
    (e) => {
        console.log(`Promise rejected. ${e}`);
    }
);

}

Promise fulfilled.


# `Promise.race()`

## `Promise.race()` expects a single array argument containing one or more _Promise_ instances
## `Promise.race()` will fulfill if and when any _Promise_ resolution is a fulfillment

In [8]:
{

const p1 = new Promise(resolve => {
    setTimeout(() => resolve('p1'), 1000);
});

const p2 = new Promise(resolve => {
    resolve('p2');
});

const p3 = Promise.race([p1, p2]).then(
    value => {
        console.log(`The first Promise to resolve: ${value}.`);
    }
);

}

The first Promise to resolve: p2.


## `Promise.race()` will reject if and when any _Promise_ resolution is a rejection

In [10]:
{

const p1 = new Promise(resolve => {
    setTimeout(() => resolve('p1'), 1000);
});

const p2 = new Promise((resolve, reject) => {
    setTimeout(() => reject('p2'), 0);
});

const p3 = Promise.race([p1, p2]).then(
    () => {},
    value => {
        console.log(`The first Promise to resolve: ${value}.`)
    }
)

}

The first Promise to resolve: p2.


## If `Promise.race()` is passed an empty array, it will never resolve

In [12]:
{

const p = Promise.race([]).then(
    () => {
        console.log('Promise fulfilled.');
    },
    () => {
        console.log('Promise rejected.');
    }
);

}

# Generators

In [1]:
{

const calculateAge = function(birthDate, delay) {
    return new Promise(resolve => {
        setTimeout(() => {
            const today = new Date();
            let age = today.getFullYear() - birthDate.getFullYear();

            if (today.getMonth() < birthDate.getMonth() || 
                today.getMonth() === birthDate.getMonth() && today.getDate() < birthDate.getDate()) {
                age--;
            }

            resolve(age);
        }, delay);
    });
};

const sayMyAge = function*(name, birthDate, delay) {
    const age = yield calculateAge(birthDate, delay);
    console.log(`${name} is ${age} years old!`);
}

const it1 = sayMyAge('Doug', new Date(1991, 1, 9), 1000);
it1.next().value.then(age => {
    it1.next(age);
});

const it2 = sayMyAge('Philippe', new Date(1991, 8, 22), 10);
it2.next().value.then(age => {
    it2.next(age);
});

}

[Promise] {}

Philippe is 25 years old!
Doug is 25 years old!
